In [1]:
import numpy as np
import sys, os
import matplotlib.pyplot as plt

sys.path.append("..")
from qgbaroclinic import OceBaroclinicModes

In [ ]:
"""
Example for computing the MEAN baroclinic modes and rossby radii of a selected region
"""

In [ ]:
# Define Ocean Baroclinic Modes object
obm = OceBaroclinicModes(x=[-160, -170], y=[40, 45])

In [ ]:
# Extract OCEAN variables form NetCDF file.
# NOTE: "latitude", "longitude" are the dimension names in NetCDF file.
current_path = os.path.dirname(__vsc_ipynb_file__)
temp, sal, depth, lat = obm.read(
    os.path.join(current_path, "../data/reanalysis/"),
    "thetao",
    "so",
    "depth",
    "latitude",
    longitude=obm.domain["x"],
    latitude=obm.domain["y"],
)
mean_lat = np.mean(lat.values)

In [ ]:
# Extract Bathymetry dataset and compute mean reagion depth
# NOTE: "lat", "lon" are the dimension names in NetCDF file.
elevation = obm.read(
    os.path.join(current_path, "../data/bathymetry/GEBCO_2023.nc"),
    "elevation",
    lat=obm.domain["y"],
    lon=obm.domain["x"],
)
mean_depth = np.abs(np.nanmean(elevation))

In [ ]:
print(f"Region mean depth {mean_depth} m, region mean lat {mean_lat} °N")

In [ ]:
# Compute pot density (you should use potential temperature!)
pot_density = obm.potential_density(temp, sal)
mean_pot_density = np.nanmean(pot_density, axis = (0,2,3))

In [13]:
obm(mean_pot_density, depth.values, mean_lat, mean_depth)

In [ ]:
# The output result is stored as attributes
rossby_rad = obm.rossby_rad
vert_structfunc = obm.vert_structfunc

In [ ]:
"rossby rad (km)", rossby_rad/1000

In [21]:
def coriolis_param(mean_lat: float) -> float:
    """
    Compute Coriolis parameter given the region mean latitude.
    """

    # earth angular velocity (1/s)
    earth_angvel = 7.29 * 1e-05
    # coriolis parameter (1/s)
    coriolis_param = 2 * earth_angvel * np.sin(mean_lat * np.pi / 180)
    return coriolis_param

In [22]:
# reminder
omega = 1.0
coriolis_parameter = coriolis_param(mean_lat)
denominator = np.sqrt(omega**2 - coriolis_parameter**2)
horizontal_wavelength = rossby_rad * (1/denominator)

In [ ]:
"horizontal wavelength (km)", horizontal_wavelength/1000

In [ ]:
plt.figure(1)
plt.plot(vert_structfunc, - obm.depth)
plt.title("Vertical Structure Function")
plt.ylabel("depth (m)")
plt.legend(["barotropic mode", "mode 1", "mode 2", "mode 3", "mode 4"])
plt.show()
plt.close()
